In [15]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1. Wczytanie danych JSON
with open('mnt/data/data.json', 'r') as file:
    data = json.load(file)

# 2. Wczytanie raportu cukrów (co 5 minut)
glucose_df = pd.read_csv('mnt/data/Michał._glucose_8-3-2025.csv', skiprows=3)  # Pomijamy nagłówki
print("Pierwsze wiersze pliku CSV:")
print(glucose_df.head())  # Debugging - sprawdzamy dane

# Ustawienie poprawnych nazw kolumn
glucose_df.columns = ['Device', 'Serial Number', 'timestamp', 'Record Type', 'glucose',
                      'Unnamed_5', 'Unnamed_6', 'Unnamed_7', 'Unnamed_8', 'Unnamed_9',
                      'Unnamed_10', 'Unnamed_11', 'Unnamed_12', 'Unnamed_13', 'Unnamed_14',
                      'Unnamed_15', 'Unnamed_16', 'Unnamed_17', 'Unnamed_18']

# Poprawne parsowanie daty
glucose_df['timestamp'] = pd.to_datetime(glucose_df['timestamp'], dayfirst=True, errors='coerce')

# Usunięcie błędnie sparsowanych wartości
glucose_df = glucose_df.dropna(subset=['timestamp'])

def get_glucose_trend(timestamp):
    """ Pobiera trend cukru po posiłku. """
    window = glucose_df[(glucose_df['timestamp'] >= timestamp) & 
                        (glucose_df['timestamp'] <= timestamp + pd.Timedelta(minutes=120))]
    if len(window) > 1:
        return window['glucose'].diff().mean()  # Średnia zmiana cukru
    return 0

# 3. Przetworzenie danych do formatu tabelarycznego
records = []
for user_id, days in data.items():
    for day in days:
        calculator_data = day.get('calculatorData')  # Pobieramy tylko raz

        if 'meals' in day and day['meals']:  # Sprawdzenie czy są posiłki
            for meal in day['meals']:
                meal_time = pd.to_datetime(calculator_data['date'], errors='coerce') if calculator_data else None
                trend = get_glucose_trend(meal_time) if meal_time else None
                
                record = {
                    'user_id': user_id,
                    'date': meal_time,
                    'meal_name': meal.get('mealName', 'Unknown'),
                    'carbs': meal.get('carbs', 0),
                    'fats': meal.get('fats', 0),
                    'proteins': meal.get('prot', 0),
                    'kcal': meal.get('kcal', 0),
                    'glucose': calculator_data['glucose'] if calculator_data else None,
                    'insulin_units': calculator_data['units']['short'] if calculator_data and 'units' in calculator_data else None,
                    'glucose_trend': trend
                }
                records.append(record)

df = pd.DataFrame(records)
df.dropna(inplace=True)  # Usuwamy brakujące wartości

# 4. Obliczanie dawki insuliny na podstawie algorytmu
def calculate_insulin(carbs, insulin_ratio):
    return carbs / 10 * insulin_ratio  # 1 WW = 10g węglowodanów

df['calculated_insulin'] = df['carbs'].apply(lambda x: calculate_insulin(x, 1))  # Załóżmy 1j na WW

# 5. Przygotowanie danych do trenowania
X = df[['carbs', 'fats', 'proteins', 'kcal', 'glucose', 'glucose_trend']].values
y = df['insulin_units'].values

# Normalizacja danych wejściowych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 6. Tworzenie modelu
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Wyjście: przewidywana korekta insuliny
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 7. Trening modelu
history = model.fit(X, y, epochs=100, batch_size=8, validation_split=0.2)

# 8. Zapisanie modelu
model.save('/mnt/data/insulin_model.h5')

# 9. Testowanie modelu
test_input = np.array([[50, 0, 0, 200, 120, 0]])  # 50g węglowodanów, 120 mg/dL glukozy, neutralny trend
test_input = scaler.transform(test_input)
test_prediction = model.predict(test_input)

# Finalna dawka = Algorytm + Korekta AI
final_dose = calculate_insulin(50, 1) + test_prediction[0][0]
print(f'Finalna przewidywana dawka insuliny: {final_dose:.2f} jednostki')


Pierwsze wiersze pliku CSV:
   FreeStyle LibreLink  B227573A-50B7-43A0-971E-11CBB6230841   
0  FreeStyle LibreLink  B227573A-50B7-43A0-971E-11CBB6230841  \
1  FreeStyle LibreLink  B227573A-50B7-43A0-971E-11CBB6230841   
2  FreeStyle LibreLink  B227573A-50B7-43A0-971E-11CBB6230841   
3  FreeStyle LibreLink  B227573A-50B7-43A0-971E-11CBB6230841   
4  FreeStyle LibreLink  B227573A-50B7-43A0-971E-11CBB6230841   

   17-09-2024 18:27  0     92  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8   
0  17-09-2024 18:42  0  111.0         NaN         NaN         NaN         NaN  \
1  17-09-2024 18:57  0  133.0         NaN         NaN         NaN         NaN   
2  17-09-2024 19:12  0  140.0         NaN         NaN         NaN         NaN   
3  17-09-2024 19:27  0  134.0         NaN         NaN         NaN         NaN   
4  17-09-2024 19:42  0  132.0         NaN         NaN         NaN         NaN   

   Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13   
0         NaN          NaN     

TypeError: Invalid comparison between dtype=datetime64[ns] and Timestamp

In [6]:
# 7. Testowanie modelu
test_input = np.array([[50, 0, 0, 200, 120]])  # 50g węglowodanów, 0 tłuszczu, 0 białka, 200 kcal, 120 mg/dL glukozy
test_input = scaler.transform(test_input)  # Normalizacja danych
test_prediction = model.predict(test_input)
print(f'Przewidywana dawka insuliny: {test_prediction[0][0]:.2f} jednostki')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Przewidywana dawka insuliny: 4.40 jednostki
